In [ ]:
# Cài các thư viện cần thiết
!pip install fastapi uvicorn nest-asyncio pyngrok python-multipart python-jose[cryptography] passlib[bcrypt]


# ----------- Python imports -----------
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from pydantic import BaseModel
from passlib.context import CryptContext
from jose import JWTError, jwt
from datetime import datetime, timedelta
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# ----------- Ngrok auth token -----------
!ngrok config add-authtoken 2vS8pjIDh8ErcG18jkv0SNtAaZD_5GsocS1ppPBeDuPSUvMC  # 👈 thay bằng token thật của bạn

# ----------- App & Config -----------
app = FastAPI()
SECRET_KEY = "smart-home-secret-key"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="/login")

# ----------- Models -----------
class User(BaseModel):
    username: str
    full_name: str
    hashed_password: str

class UserCreate(BaseModel):
    username: str
    password: str
    full_name: str

class Token(BaseModel):
    access_token: str
    token_type: str

# ----------- Fake DB -----------
fake_users_db = {}

# ----------- Auth Utils -----------
def get_password_hash(password):
    return pwd_context.hash(password)

def verify_password(plain_password, hashed_password):
    return pwd_context.verify(plain_password, hashed_password)

def create_access_token(data: dict, expires_delta: timedelta = None):
    to_encode = data.copy()
    expire = datetime.utcnow() + (expires_delta or timedelta(minutes=15))
    to_encode.update({"exp": expire})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

def get_user(username: str):
    user = fake_users_db.get(username)
    return User(**user) if user else None

def authenticate_user(username: str, password: str):
    user = get_user(username)
    if not user or not verify_password(password, user.hashed_password):
        return False
    return user

def get_current_user(token: str = Depends(oauth2_scheme)):
    credentials_exception = HTTPException(
        status_code=status.HTTP_401_UNAUTHORIZED,
        detail="Không thể xác thực token.",
        headers={"WWW-Authenticate": "Bearer"},
    )
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username = payload.get("sub")
        if username is None:
            raise credentials_exception
    except JWTError:
        raise credentials_exception
    user = get_user(username)
    if user is None:
        raise credentials_exception
    return user

# ----------- API Routes -----------

@app.get("/")
def root():
    return {"message": "🚀 Hệ thống điều khiển nhà thông minh đã sẵn sàng!"}

@app.post("/register")
def register(user: UserCreate):
    if user.username in fake_users_db:
        raise HTTPException(status_code=400, detail="Tên người dùng đã tồn tại.")
    fake_users_db[user.username] = {
        "username": user.username,
        "full_name": user.full_name,
        "hashed_password": get_password_hash(user.password),
    }
    return {"msg": "✅ Đăng ký thành công!"}

@app.post("/login", response_model=Token)
def login(form_data: OAuth2PasswordRequestForm = Depends()):
    user = authenticate_user(form_data.username, form_data.password)
    if not user:
        raise HTTPException(status_code=400, detail="Sai tài khoản hoặc mật khẩu")
    access_token = create_access_token(data={"sub": user.username})
    return {"access_token": access_token, "token_type": "bearer"}

@app.get("/device/control")
def control_device(current_user: User = Depends(get_current_user)):
    return {"msg": f"🎮 Thiết bị được điều khiển bởi: {current_user.full_name}"}

@app.post("/device/on")
def turn_on_device(current_user: User = Depends(get_current_user)):
    # Ở đây bạn có thể tích hợp gửi tín hiệu bật thiết bị thật
    return {"msg": f"💡 Thiết bị đã được BẬT bởi {current_user.full_name}"}

@app.post("/device/off")
def turn_off_device(current_user: User = Depends(get_current_user)):
    # Ở đây bạn có thể tích hợp gửi tín hiệu tắt thiết bị thật
    return {"msg": f"🔌 Thiết bị đã được TẮT bởi {current_user.full_name}"}

# ----------- Chạy ngrok & app -----------
public_url = ngrok.connect(8000)
print("🔗 Public URL:", public_url)

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🔗 Public URL: NgrokTunnel: "https://ac1b-34-145-182-209.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [257]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     58.186.75.88:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     58.186.75.88:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     58.186.75.88:0 - "POST /login HTTP/1.1" 400 Bad Request
INFO:     58.186.75.88:0 - "POST /login HTTP/1.1" 400 Bad Request
INFO:     58.186.75.88:0 - "POST /register HTTP/1.1" 200 OK
INFO:     58.186.75.88:0 - "POST /login HTTP/1.1" 200 OK
INFO:     58.186.75.88:0 - "POST /login HTTP/1.1" 200 OK
INFO:     58.186.75.88:0 - "GET /device/control HTTP/1.1" 200 OK
INFO:     58.186.75.88:0 - "POST /device/on HTTP/1.1" 200 OK
INFO:     58.186.75.88:0 - "POST /device/off HTTP/1.1" 200 OK
